# Network simulation

To understand the human brain, we need to analyze its signals. As the brain is hidden within the skull, there is a limited amount of methods available to extract brain signals (in the living human). We mostly use fMRI and EEG and we learn mostly about fMRI here. 

In [ ]:
## Recap

### Part 1: Connectivity 
# Learning goals
- define what nodes and edges are
- describe basic graph theory characteristics of a brain network
- list the steps on how get from a brain MRI scan to a brain network
- explain how different connectivity characteristics influence the resulting dynamics

## Lecture

### Graph theory basics

![alt text](figures/network_02.jpg "Traffic in Ruhrgebiet")
Consider traffic as a sort of information transfer.

![alt text](figures/network_01.jpg "Nodes_Edges")

Ref: Handiru (2021)

In the context of brain networks in this study, every brain region is a node, whereas the connection between nodes is an edge. Colored shaded regions correspond to modules which are groups of interconnected nodes but have fewer connections to other modules. On the left, the concept of network segregation is presented as a network comprising of multiple modules (or segregated subnetworks) whereas, on the right, the network integration is illustrated as a group of distant regions interconnected by long‐range connections via connector hubs (nodes with a high degree of connectivity). The choice of nodes and edges can be influenced by the anatomical parcellation schemes and measures for determining connectivity. Some of the connections are direct, while others are indirect (to pass from one to another, the signal has to travel via another node).

![alt text](figures/network_04.jpg "Between regularity and chaos")

Ref: Radiologykey.com

Why should we apply graph theory to the brain? (according to Kaiser et al. 2011)
1. Networks provide an abstraction that can reduce the complexity when dealing with neural networks. 
2. The overall organization of brain networks has been proven reliable in that features such as modularity, present but varying to some degree, could be found in all human brain networks. 
3. Using the same frame of reference, given by the identity of network nodes as representing brain regions, both comparisons between subjects as well as comparisons of different kinds of networks (e.g. structural versus functional) are feasible (Rubinov and Sporns, 2010).

<div class="alert alert-block alert-success">
<b>Exercise 1 </b><p>
Steps to calculate structural and functional connectivity networks (i.e., connectomes)
    
<p>
 -End of exercise-
    </div>

In [ ]:
# setting parameters
N = 6
# cmat is the structural connectivity matrix (here we start with a fully
# connected network)
cmat = np.ones((N, N)) # fully connected network
# dmat is the delay matrix, at the moment we consider instantaneous interactions
# between brain regions, so no delays
dmat = np.zeros((N,N)) # no delays

# Let's create a network model!
network_model = KuramotoModel(Cmat=cmat, Dmat=dmat)
# Set the duration of the simulation
network_model.params['duration'] = 50
# Set the intrinsic frequency
network_model.params['omega'] = np.random.rand((N)) * 0.25 * np.pi
# Let's start without any noise
network_model.params['sigma_ou'] = 0.
# For now our oscillators are uncoupled, so K = 0
network_model.params['k'] = 0
network_model.run()

theta = network_model['theta'].T
# cap the phase to be between 0 and 2pi
theta_capped = np.mod(theta, 2*np.pi)

# set up the figure
fig, ax = plt.subplots(1, 1, figsize=(10, 4))

plt.plot(network_model.t, theta_capped)
plt.xlabel("Time")
plt.ylabel("Theta")
plt.yticks(np.arange(0, 2*np.pi+0.1, np.pi/2), [ r"$0$", r"$\pi/2$", r"$\pi$", r"$3/4\pi$", r"$2\pi$",])# modify y-axis ticks to be in multiples of pi
plt.show()

To understand the effect of connectivity on model dynamics, we will use an adaptation of the Hopf model/ Stuart-Landau model. The model simulates the activity of all cortical ROIs, with their intercommunication determined by the empirical structural connectivity (SC) matrix, $A$. 
The elements of the connectivity matrix are $C_{ij} = 0$ if no fiber bundle exist projecting from ROI $i$ to ROI $j$.
 Otherwise, $A_{ij}$ represent the weight of that connection. See Ref. [1] for a detailed description of the complete model and its resulting spatio-temporal dynamics.

The dynamics of a single Stuart-Landau system [2,3] is given by the following differential equation, with complex variable $z$:

$\dot{z} \;=\; (a + i\omega)\,z \;+\; z \, |z^2|$

We will not consider the bifurcation parameter further and set it to zero. $\omega$ the natural frequency of oscillation. 

The dynamics of each ROI will follow the equation above, coupled via the connectivity matrix. Each region is provided with an additive Guassian noise $\xi(t)$ of standard deviation $\beta$. The activity of each ROI in the complete system is described by:

$\dot{z}_j \;=\; (a_j + i\omega_j)\,z_j \;+\; z_j \, |z_j^2| \;+\; g \sum_{j=1}^N C_{ij} (z_i - z_j) \;+\; \beta \xi_j(t)$.

As you learned in the previous lectures, $g$ is a global coupling parameter that re-scales the weights of every connection. It controls the level of synchrony of the coupled system. When $g=0$, the system is uncoupled and all ROIs behave independently. 
As $g$ increases, the interdependence between ROIs becomes stronger and may lead to a global synchronised state.

--------------
[1] G. Deco, M.L. Kringelbach, B.K. Jirsa & P. Ritter, "The dynamics of resting fluctuations in the brain: metastability and its dynamical core." *Sci. Reps.* **7**:3095 (2017).

[2] L.D. Landau, "On the problem of turbulence". *Dokl. Akad. Nauk SSSR* **44**(8), 339-349  (1944).

[3] J.T. Stuart, "On the non-linear mechanics of hydrodynamic stability". *J. Fluid Mechcs* **4**(1), 1-21 (1958).

In [ ]:
# Let's import all the relevant libraries
from neurolib.models.hopf import HopfModel
import numpy as np
import matplotlib.pyplot as plt
from nilearn.connectome import ConnectivityMeasure


<div class="alert alert-block alert-success">
<b>Exercise 2: effects of connectivity on brain dynamics</b><p>
We will start with the adapted Hopf model and explore how the connectivity influences the dynamics. Adapt the code below. You can also copy the code first and adjust the code for each of the six exercises. 

1. Let's first simulate the the fully interconnected network. Adjust the structural connectivity matrix in the code below so that all nodes are fully connected.

ANSWER: ...

2. Now, what happens if a region becomes fully disconnected? 

ANSWER: ...

3. What happens if you change the connections to slightly less than 1, i.e. 0.9?

ANSWER: ...

4. Repeat all the above steps but now change the coupling parameter to 0, 0.5 and 1. How does the coupling parameter influence the dynamics? 

ANSWER: ...

5. Create a structural connectivity matrix that is not homogeneous, i.e., the nodes are not equally connected with each other. Create a completely random network first (e.g., using the np.random function). The self-connections along the diagonal should be between 0 and 1 and the self-connections (along the diagonal) should be 1. What are the effects on the time series and on functional connectivity?

ANSWER: ...

6. Now, create two modules within the network that are connected by one node. Please draw the matrix first that you want to create and transfer it to code. How do the time series from one (random) node of one community and one (random) node of the other community behave in dependance of the coupling and the connector hub connectivity? 

ANSWER: ...

   
<p>
 -End of exercise-
 </div>

In [ ]:
# functional connectivity measure
correlation_measure = ConnectivityMeasure(kind='correlation')

# We import the model
model = HopfModel()

# Set noise strength to zero to define target state
model.params.sigma_ou = 0

# setting parameters
# we have 10 brain regions
N = 10
# cmat is the structural connectivity matrix (here we start with a fully
# connected network)
cmat = ....
# dmat is the delay matrix, at the moment we consider instantaneous interactions
# between brain regions, so no delays
dmat = np.zeros((N,N)) # no delays

# intrinsic angular frequency of the oscillation
model.params['omega'] = np.ones((N)) * 0.25 * np.pi
# set the noise here
model.params['sigma_ou'] = 0
# set the coupling here
model.params['k'] = 1
# duration of the oscillation
model.params['duration'] = 1.0*1000


model.run()

plt.figure(figsize=(4, 4))
plt.plot(model.t, model.x.T, c='k', lw=2)
plt.xlabel("t [ms]")
plt.ylabel("Activity")
plt.title(f"$a$: {bif_param}\n $\omega$: {frequency}")
plt.grid(True)
plt.show()

plt.figure(figsize=(4, 4))
functional_connectivity = correlation_measure.fit_transform([model.x.T])
plt.imshow(np.squeeze(functional_connectivity))

In [ ]:
# solutions
(maybe QR code?)

## Check your learning goals!
# Part 2: Fitting
## Learning goals
<input type="checkbox" id="1" name="1" value="yes">
<label for="vehicle1">I can define what nodes and edges are</label><br>
<input type="checkbox" id="2" name="2" value="yes">
<label for="vehicle1">I can describe basic graph theory characteristics of a brain network</label><br>
<input type="checkbox" id="3" name="3" value="yes">
<label for="vehicle1">I can list the steps on how get from a brain MRI scan to a brain network</label><br>
<input type="checkbox" id="4" name="4" value="yes">
<label for="vehicle1">I can explain how different connectivity characteristics influence the resulting dynamics</label><br>

# Part 2: Fitting
Imagine you have a dataset and you want to simulate the time series. How do you make sure that the parameters are chosen in such a way that they represent the data best? 

In [ ]:
# functional connectivity measure
correlation_measure = ConnectivityMeasure(kind='correlation')

# We import the model
model = HopfModel()

# Set noise strength to zero to define target state
model.params.sigma_ou = 0

# setting parameters
# we have 10 brain regions
N = 10
# cmat is the structural connectivity matrix (here we start with a fully
# connected network)
cmat = ....
# dmat is the delay matrix, at the moment we consider instantaneous interactions
# between brain regions, so no delays
dmat = np.zeros((N,N)) # no delays

# intrinsic angular frequency of the oscillation
model.params['omega'] = np.ones((N)) * 0.25 * np.pi
# set the noise here
model.params['sigma_ou'] = 0
# set the coupling here
model.params['k'] = 1
# duration of the oscillation
model.params['duration'] = 1.0*1000

model.run()

plt.figure(figsize=(4, 4))
plt.plot(model.t, model.x.T, c='k', lw=2)
plt.xlabel("t [ms]")
plt.ylabel("Activity")
plt.title(f"$a$: {bif_param}\n $\omega$: {frequency}")
plt.grid(True)
plt.show()

plt.figure(figsize=(4, 4))
functional_connectivity = correlation_measure.fit_transform([model.x.T])
plt.imshow(np.squeeze(functional_connectivity))

Further reading material
Olaf Sporns. Networks of the Brain. MIT Press, 2010